In [12]:
import pandas as pd
import numpy as np
from WrongData import introduce_filthy_data
from sklearn.impute import KNNImputer

## Adding filthy data

In [16]:
df = pd.read_csv("./dataset/spotify_songs_restructured.csv", sep=",")
new_data = introduce_filthy_data(df, 1500)
new_data.to_csv("./dataset/spotify_songs_restructured_filthy.csv", index=False)
new_data

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6.0,-2.634,1.0,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754.0
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,11.0,-4.969,1.0,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600.0
2,All the Time - Don Diablo Remix,Zara Larsson,70.0,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1.0,-3.432,0.0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616.0
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,1.0,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33328,Goza,Henry Mendez,45.0,Goza,2020-01-03,Tropical Rising,latin,tropical,0.792,0.904,10.0,-4.707,0.0,0.0818,0.0994,0.000000,0.0908,0.651,95.011,184421.0
33329,Everybody,Logic,46.0,Everybody,2017-03-31,"🔥💵 Hip Hop, Rap, Heavy 808's - New School",rap,southern hip hop,0.885,0.936,1.0,-5.879,1.0,0.0894,0.1480,0.000000,0.0703,0.763,110.011,162373.0
33330,Streetcar,Daniel Caesar,71.0,Pilgrim's Paradise,2015-11-12,NEO-soul,r&b,neo soul,0.410,0.298,3.0,-11.796,1.0,0.0473,0.7210,0.000000,0.1580,0.142,109.651,246972.0
33331,Hundred Miles,Yall,64.0,Hundred Miles,2015-10-16,Tropical Morning,latin,tropical,0.479,0.714,7.0,-3.776,0.0,0.1090,0.0250,0.015300,0.2060,0.490,98.230,172413.0


## Transforming the data

In [3]:
df = pd.read_csv('dataset/spotify_songs_restructured.csv', sep=',')
df = df.fillna('Unknown')
df = df[df['track_name'] != 'Unknown']
df = df.drop_duplicates(inplace=False)
df.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000,32828.000000
mean,42.483551,0.654850,0.698603,5.373949,-6.719529,0.565737,0.107053,0.175352,0.084760,0.190175,0.510556,120.883642,225796.829779
std,24.980476,0.145092,0.180916,3.611572,2.988641,0.495667,0.101307,0.219644,0.224245,0.154313,0.233152,26.903632,59836.492346
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.581000,2.000000,-8.171250,0.000000,0.041000,0.015100,0.000000,0.092700,0.331000,99.961000,187804.500000
50%,45.000000,0.672000,0.721000,6.000000,-6.166000,1.000000,0.062500,0.080400,0.000016,0.127000,0.512000,121.984000,216000.000000
75%,62.000000,0.761000,0.840000,9.000000,-4.645000,1.000000,0.132000,0.255000,0.004830,0.248000,0.693000,133.918250,253581.250000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000


## Removing outliers

In [85]:
# Define conditions for outliers based on duration and tempo
Q10_custom = df['duration_ms'].quantile(0.05)
Q90_custom = df['duration_ms'].quantile(0.95)

Q10_custom_tempo = df['tempo'].quantile(0.05)
Q90_custom_tempo = df['tempo'].quantile(0.95)

# Remove outliers
filtered_df = df[(df['duration_ms'] > Q10_custom) & (df['duration_ms'] < Q90_custom)]
filtered_df = filtered_df[(filtered_df['tempo'] > Q10_custom_tempo) & (filtered_df['tempo'] < Q90_custom_tempo)]
filtered_df.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000,26637.000000
mean,42.954349,0.663531,0.711139,5.360326,-6.504109,0.565116,0.100966,0.164552,0.070301,0.190661,0.514948,119.661413,221796.255547
std,24.964623,0.139289,0.172943,3.608796,2.835029,0.495751,0.094621,0.207678,0.201570,0.153664,0.231683,21.629617,41796.969677
min,0.000000,0.077100,0.000175,0.000000,-46.448000,0.000000,0.022400,0.000001,0.000000,0.009360,0.000010,81.076000,148400.000000
25%,25.000000,0.573000,0.597000,2.000000,-7.882000,0.000000,0.040300,0.014800,0.000000,0.092600,0.335000,100.418000,190453.000000
50%,46.000000,0.677000,0.733000,6.000000,-5.977000,1.000000,0.060100,0.075800,0.000013,0.128000,0.517000,121.035000,215556.000000
75%,63.000000,0.766000,0.847000,9.000000,-4.549000,1.000000,0.121000,0.235000,0.002970,0.250000,0.696000,130.041000,247907.000000
max,100.000000,0.981000,1.000000,11.000000,1.275000,1.000000,0.918000,0.989000,0.994000,0.994000,0.991000,173.946000,337333.000000


## Categorical Encoding

In [86]:
# Define categories and labels for track_popularity
popularity_bins = [0, 20, 40, 60, 80, 100]
popularity_labels = ['Very Low Popularity', 'Low Popularity', 'Medium Popularity', 'High Popularity', 'Very High Popularity']
filtered_df['popularity_category'] = pd.Categorical(pd.cut(filtered_df['track_popularity'], bins=popularity_bins, labels=popularity_labels))

# Define categories and labels for tempo
tempo_bins = [0, 60, 90, 120, 150, 180, float('inf')]
tempo_labels = ['Very Slow', 'Slow', 'Moderate', 'Fast', 'Very Fast', 'Extremely Fast']
filtered_df['tempo_category'] = pd.Categorical(pd.cut(filtered_df['tempo'], bins=tempo_bins, labels=tempo_labels))

# Define categories and labels for loudness
loudness_bins = [-float('inf'), -15, -10, -5, 0, float('inf')]
loudness_labels = ['Very Quiet', 'Quiet', 'Moderate', 'Loud', 'Very Loud']
filtered_df['loudness_category'] = pd.Categorical(pd.cut(filtered_df['loudness'], bins=loudness_bins, labels=loudness_labels))
filtered_df

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,...,0.0583,0.10200,0.000000,0.0653,0.518,122.036,194754,High Popularity,Fast,Loud
1,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,...,0.0373,0.07240,0.004210,0.3570,0.693,99.972,162600,High Popularity,Moderate,Loud
2,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,...,0.0742,0.07940,0.000023,0.1100,0.613,124.008,176616,High Popularity,Fast,Loud
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,...,0.1020,0.02870,0.000009,0.2040,0.277,121.956,169093,Medium Popularity,Fast,Loud
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,...,0.0359,0.08030,0.000000,0.0833,0.725,123.976,189052,High Popularity,Fast,Loud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32825,Booyah - Radio Edit,Showtek,47,Booyah,2013-01-01,♥ EDM LOVE 2020,edm,progressive electro house,0.559,0.916,...,0.0626,0.04530,0.000013,0.2250,0.195,128.012,215295,Medium Popularity,Fast,Loud
32826,Wasted,Tiësto,47,Wasted,2014-04-22,♥ EDM LOVE 2020,edm,progressive electro house,0.645,0.832,...,0.0294,0.00106,0.002640,0.1990,0.375,112.028,188371,Medium Popularity,Moderate,Moderate
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27,Many Ways,2013,♥ EDM LOVE 2020,edm,progressive electro house,0.581,0.640,...,0.0365,0.02660,0.000000,0.5720,0.288,128.001,196993,Low Popularity,Fast,Moderate
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922,...,0.0936,0.07660,0.000000,0.0668,0.210,128.170,204375,Medium Popularity,Fast,Loud


## Feature splitting for release_date (Domain expertise)

In [87]:
# Convert track_album_release_date to datetime, coercing errors
filtered_df['track_album_release_date'] = pd.to_datetime(filtered_df['track_album_release_date'], errors='coerce')

# Extract the month name and create a new column
filtered_df['release_month'] = filtered_df['track_album_release_date'].dt.strftime('%B')

# Display the DataFrame
filtered_df.drop(columns=['instrumentalness', 'acousticness', 'mode', 'playlist_subgenre', 'playlist_name'], inplace=True)
filtered_df

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748,0.916,6,-2.634,0.0583,0.0653,0.518,122.036,194754,High Popularity,Fast,Loud,June
1,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815,11,-4.969,0.0373,0.3570,0.693,99.972,162600,High Popularity,Moderate,Loud,December
2,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931,1,-3.432,0.0742,0.1100,0.613,124.008,176616,High Popularity,Fast,Loud,July
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,2019-07-19,pop,0.718,0.930,7,-3.778,0.1020,0.2040,0.277,121.956,169093,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650,0.833,1,-4.672,0.0359,0.0833,0.725,123.976,189052,High Popularity,Fast,Loud,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32825,Booyah - Radio Edit,Showtek,47,Booyah,2013-01-01,edm,0.559,0.916,11,-3.050,0.0626,0.2250,0.195,128.012,215295,Medium Popularity,Fast,Loud,January
32826,Wasted,Tiësto,47,Wasted,2014-04-22,edm,0.645,0.832,2,-5.595,0.0294,0.1990,0.375,112.028,188371,Medium Popularity,Moderate,Moderate,April
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27,Many Ways,NaT,edm,0.581,0.640,5,-8.367,0.0365,0.5720,0.288,128.001,196993,Low Popularity,Fast,Moderate,NaN
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,City Of Lights (Vocal Mix),2014-04-28,edm,0.428,0.922,2,-1.814,0.0936,0.0668,0.210,128.170,204375,Medium Popularity,Fast,Loud,April


## Dropping rows with missing track_album_release_date

In [88]:
filtered_df = filtered_df.dropna()
filtered_df.isna().sum()

track_name                  0
track_artist                0
track_popularity            0
track_album_name            0
track_album_release_date    0
playlist_genre              0
danceability                0
energy                      0
key                         0
loudness                    0
speechiness                 0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
popularity_category         0
tempo_category              0
loudness_category           0
release_month               0
dtype: int64

## Saving the cleaned dataset

In [89]:
filtered_df.to_csv('dataset/spotify_songs_cleaned.csv', index=False)
filtered_df

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748,0.916,6,-2.634,0.0583,0.0653,0.518,122.036,194754,High Popularity,Fast,Loud,June
1,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815,11,-4.969,0.0373,0.3570,0.693,99.972,162600,High Popularity,Moderate,Loud,December
2,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931,1,-3.432,0.0742,0.1100,0.613,124.008,176616,High Popularity,Fast,Loud,July
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,2019-07-19,pop,0.718,0.930,7,-3.778,0.1020,0.2040,0.277,121.956,169093,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650,0.833,1,-4.672,0.0359,0.0833,0.725,123.976,189052,High Popularity,Fast,Loud,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32824,Hey Brother,Avicii,20,True,2013-01-01,edm,0.545,0.780,7,-4.867,0.0436,0.0828,0.458,125.014,255093,Very Low Popularity,Fast,Loud,January
32825,Booyah - Radio Edit,Showtek,47,Booyah,2013-01-01,edm,0.559,0.916,11,-3.050,0.0626,0.2250,0.195,128.012,215295,Medium Popularity,Fast,Loud,January
32826,Wasted,Tiësto,47,Wasted,2014-04-22,edm,0.645,0.832,2,-5.595,0.0294,0.1990,0.375,112.028,188371,Medium Popularity,Moderate,Moderate,April
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,City Of Lights (Vocal Mix),2014-04-28,edm,0.428,0.922,2,-1.814,0.0936,0.0668,0.210,128.170,204375,Medium Popularity,Fast,Loud,April
